In [1]:
import os
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm
from tensorflow.keras import layers
from PIL import Image

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [2]:
train_csv_path = './data/train.csv'
valid_csv_path = './data/val.csv'
train_df = pd.read_csv(train_csv_path)
valid_df = pd.read_csv(valid_csv_path)

In [3]:
label_map = {}
for idx, label in enumerate(train_df['labels'].unique()):
    label_map[label] = idx
label_map

{'화남/불쾌': 0, '공포': 1, '행복/즐거움': 2, '편안/안정': 3, '공격성': 4, '불안/슬픔': 5}

In [4]:
train_df['labels'] = train_df['labels'].apply(lambda x: label_map[x])
valid_df['labels'] = valid_df['labels'].apply(lambda x: label_map[x])

In [5]:
train_df

,image_path,bbox,labels
0,/media/jaeho/SSD/datasets/pet_data/train/cat/i...,"401,441,431,435",0
1,/media/jaeho/SSD/datasets/pet_data/train/cat/i...,"358,447,472,443",0
2,/media/jaeho/SSD/datasets/pet_data/train/cat/i...,"358,445,472,445",0
3,/media/jaeho/SSD/datasets/pet_data/train/cat/i...,"428,183,704,749",0
4,/media/jaeho/SSD/datasets/pet_data/train/cat/i...,"512,22,621,880",0
...,...,...,...
2127943,/media/jaeho/SSD/datasets/pet_data/train/dog/i...,"412,43,1096,1011",2
2127944,/media/jaeho/SSD/datasets/pet_data/train/dog/i...,"524,102,1094,949",2
2127945,/media/jaeho/SSD/datasets/pet_data/train/dog/i...,"541,410,974,550",2
2127946,/media/jaeho/SSD/datasets/pet_data/train/dog/i...,"517,342,976,532",2


In [6]:
train_df

,image_path,bbox,labels
0,/media/jaeho/SSD/datasets/pet_data/train/cat/i...,"401,441,431,435",0
1,/media/jaeho/SSD/datasets/pet_data/train/cat/i...,"358,447,472,443",0
2,/media/jaeho/SSD/datasets/pet_data/train/cat/i...,"358,445,472,445",0
3,/media/jaeho/SSD/datasets/pet_data/train/cat/i...,"428,183,704,749",0
4,/media/jaeho/SSD/datasets/pet_data/train/cat/i...,"512,22,621,880",0
...,...,...,...
2127943,/media/jaeho/SSD/datasets/pet_data/train/dog/i...,"412,43,1096,1011",2
2127944,/media/jaeho/SSD/datasets/pet_data/train/dog/i...,"524,102,1094,949",2
2127945,/media/jaeho/SSD/datasets/pet_data/train/dog/i...,"541,410,974,550",2
2127946,/media/jaeho/SSD/datasets/pet_data/train/dog/i...,"517,342,976,532",2


In [7]:
target = train_df.pop('labels')
target_valid = valid_df.pop('labels')
target

0          0
1          0
2          0
3          0
4          0
          ..
2127943    2
2127944    2
2127945    2
2127946    2
2127947    2
Name: labels, Length: 2127948, dtype: int64

In [8]:
train_df

,image_path,bbox
0,/media/jaeho/SSD/datasets/pet_data/train/cat/i...,"401,441,431,435"
1,/media/jaeho/SSD/datasets/pet_data/train/cat/i...,"358,447,472,443"
2,/media/jaeho/SSD/datasets/pet_data/train/cat/i...,"358,445,472,445"
3,/media/jaeho/SSD/datasets/pet_data/train/cat/i...,"428,183,704,749"
4,/media/jaeho/SSD/datasets/pet_data/train/cat/i...,"512,22,621,880"
...,...,...
2127943,/media/jaeho/SSD/datasets/pet_data/train/dog/i...,"412,43,1096,1011"
2127944,/media/jaeho/SSD/datasets/pet_data/train/dog/i...,"524,102,1094,949"
2127945,/media/jaeho/SSD/datasets/pet_data/train/dog/i...,"541,410,974,550"
2127946,/media/jaeho/SSD/datasets/pet_data/train/dog/i...,"517,342,976,532"


In [9]:
def preprocess(x, y):
    bbox = tf.strings.split(x[1], ",")
    bbox = tf.strings.to_number(bbox, out_type=tf.int32)
    img = tf.io.read_file(x[0])
    img = tf.image.decode_and_crop_jpeg(img, bbox, channels=0)
    img /= 255
    img = tf.image.resize(img, [224, 224])
    img.set_shape([224, 224, 3])

    # label smoothing 적용?
    label = tf.one_hot(y, 6)

    return img, label


In [10]:
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, 0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    return image, label

In [11]:
BATCH_SIZE = 64
DATASET_SIZE = len(train_df)

---

In [12]:
train_ds = tf.data.Dataset.from_tensor_slices((train_df.values, target.values))
train_ds = train_ds.map(preprocess, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.map(augment, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.batch(BATCH_SIZE)
train_ds = train_ds.shuffle(10)
train_ds = train_ds.repeat()
train_ds = train_ds.prefetch(AUTOTUNE)

In [13]:
valid_ds = tf.data.Dataset.from_tensor_slices((valid_df.values, target_valid.values))
valid_ds = valid_ds.map(preprocess, num_parallel_calls=AUTOTUNE)
valid_ds = valid_ds.batch(BATCH_SIZE)
valid_ds = valid_ds.prefetch(AUTOTUNE)

In [14]:
INPUT_SHAPE = (224, 224, 3)

In [15]:
base_model = tf.keras.applications.EfficientNetB0(input_shape=INPUT_SHAPE,
                                                    include_top=False, weights='imagenet',
                                                    classes=6)

In [16]:
base_model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization[0][0]              
_____________________________________________________________________________________

In [17]:
gap_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(512, activation='relu')
prediction_layer = tf.keras.layers.Dense(6, activation='softmax')

model = tf.keras.Sequential([
    base_model,
    gap_layer,
    dense_layer,
    prediction_layer
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, 7, 7, 1280)        4049571   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               655872    
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 3078      
Total params: 4,708,521
Trainable params: 4,666,498
Non-trainable params: 42,023
_________________________________________________________________


In [18]:
model_checkpoint_dir_path = '/media/jaeho/SSD/ai_pbl/checkpoints/'

In [19]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
mc = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_checkpoint_dir_path+"{epoch}-{val_loss:.2f}-{val_accuracy:.2f}.h5",
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

lr = 0.001, default

In [20]:
lr = 0.0001

In [21]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = lr),
    loss = tf.keras.losses.categorical_crossentropy,
    metrics = ['accuracy']
)

In [22]:
history = model.fit(
    train_ds,
    epochs=100,
    validation_data = valid_ds,
    steps_per_epoch = DATASET_SIZE//BATCH_SIZE,
    callbacks = [es, mc]
)

Epoch 1/100
33249/33249 [==============================] - 15766s 474ms/step - loss: 0.7524 - accuracy: 0.7239 - val_loss: 1.5947 - val_accuracy: 0.2901

Epoch 00001: val_accuracy improved from -inf to 0.29014, saving model to /media/jaeho/SSD/ai_pbl/checkpoints/1-1.59-0.29.h5


/home/jaeho/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/100
33249/33249 [==============================] - 15782s 475ms/step - loss: 0.7331 - accuracy: 0.7280 - val_loss: 1.2133 - val_accuracy: 0.3025

Epoch 00002: val_accuracy improved from 0.29014 to 0.30253, saving model to /media/jaeho/SSD/ai_pbl/checkpoints/2-1.21-0.30.h5
Epoch 3/100
33249/33249 [==============================] - 15797s 475ms/step - loss: 0.7129 - accuracy: 0.7331 - val_loss: 1.8033 - val_accuracy: 0.3055

Epoch 00003: val_accuracy improved from 0.30253 to 0.30549, saving model to /media/jaeho/SSD/ai_pbl/checkpoints/3-1.80-0.31.h5
Epoch 4/100
33249/33249 [==============================] - 15800s 475ms/step - loss: 0.6911 - accuracy: 0.7384 - val_loss: 1.2806 - val_accuracy: 0.2906

Epoch 00004: val_accuracy did not improve from 0.30549
Epoch 5/100
33249/33249 [==============================] - 15863s 477ms/step - loss: 0.6611 - accuracy: 0.7455 - val_loss: 1.3636 - val_accuracy: 0.2906

Epoch 00005: val_accuracy did not improve from 0.30549
Epoch 6/100
 1671/33

KeyboardInterrupt: 